In [5]:
import pandas as pd
import subprocess
import numpy as np
import ipywidgets as widgets
try:
    import wget
except:
    !pip install wget
    import wget
file_name = "All_decay_modes_power_time_series_1001_steps.csv"
dir = subprocess.os.getcwd() + "\\"
path = dir + file_name
if subprocess.os.path.exists(path):
    print("Importing ", file_name)
    df = pd.read_csv(path)
elif subprocess.os.path.exists("C:\\Cross-Section-Data\\" + file_name):
    print("Importing ", file_name)
    df = pd.read_csv("C:\\Cross-Section-Data\\" + file_name)
else: 
    url = "https://github.com/MarcosP7635/power_densities/raw/main/All_decay_modes_power_time_series_1001_steps.zip"
    print("Downloading Data from ", url)
    file_name = "All_decay_modes_power_time_series_1001_steps.zip"
    path = subprocess.os.getcwd() + "\\" + file_name
    wget.download(url, path)
    print("Importing ", path)
    df = pd.read_csv(path)
df

Importing  All_decay_modes_power_time_series_1001_steps.csv


1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10  \
0                                          0.000000e+00                                      
1                                          0.000000e+00                                      
2                                          0.000000e+00                                      
3                                          0.000000e+00                                      
4                                          0.000000e+00                                      
...                                                 ...                                      
3084                                       1.059881e+11                                      
3085                                       5.397544e+10                                      
3086                                       0.000000e+00                                      
3087                                       0.000000e+00                                      
3088                                       2.141764e+12                                      

      1.047128548050898505549645783823682271757193893433574137820153812424537637016191e-10  \
0                                          0.000000e+00                                      
1                                          0.000000e+00                                      
2                                          0.000000e+00                                      
3                                          0.000000e+00                                      
4                                          0.000000e+00                                      
...                                                 ...                                      
3084                                       1.059881e+11                                      
3085                                       5.397544e+10                                      
3086                                       0.000000e+00                                      
3087                                       0.000000e+00                                      
3088                                       2.141764e+12                                      

      1.0964781961431828604257317209264159331313714105011702911985913060102831306613e-10  \
0                                          0.000000e+00                                    
1                                          0.000000e+00                                    
2                                          0.000000e+00                                    
3                                          0.000000e+00                                    
4                                          0.000000e+00                                    
...                                                 ...                                    
3084                                       1.059881e+11                                    
3085                                       5.397544e+10                                    
3086                                       0.000000e+00                                    
3087                                       0.000000e+00                                    
3088                                       2.141764e+12                                    

      1.148153621496884066480156540139268838588675605873626387416861363160599339507005e-10  \
0                                          0.000000e+00                                      
1                                          0.000000e+00                                      
2                                          0.000000e+00                                      
3                                          0.000000e+00                                      
4                                          0.000000e+00                                      
...                                                 ...                        

In [23]:
df.set_index('Parent Isotope', inplace = True)
time_array = [float(t) for t in df.columns]

In [24]:
def get_max(entries):
  cols = [''.join([l for l in entry if l.isdigit()]) + 
          ''.join([l for l in entry if l.isalpha()])
          for entry in entries]#.split(",")]
  max_y = np.max(df.loc[cols[0]])
  for col in cols[1:]:
    max_y = max(max_y, np.max(df.loc[col]))
  if max_y == 0 or max_y < 0:
    return 1
  return max_y 
import plotly.express as px
import plotly.graph_objects as go
def plot(entries):
  fig = go.Figure()
  for entry in entries:#.split(","):
    A = int(''.join([l for l in entry if l.isdigit()]))
    El = ''.join([l for l in entry if l.isalpha()])
    entry = str(A) + El
    if df.loc[entry][0] == 0.0 :
        print(entry, " has insufficient data")
    else:
        fig.add_trace(go.Scatter(x = time_array,
            y = df.loc[entry] / A,
            name = El + "-" + str(A), mode = "markers"))
  y_dict = dict(type = "log", title = "Power (Watts)", nticks = 16,
          range = [-6, np.log10(get_max(entries))] )
  x_dict = dict(type = "log", title = "Time (Seconds)", nticks = 24,
                range = [-10, 10])
  if len(entries) > 1 :
    title = "Power Produced by 1 Gram Each of " + ','.join(entries)
  else:
    title = "Power Produced by 1 Gram of " + entries[0]
  fig.update_layout(xaxis = x_dict, yaxis = y_dict,
          title = title)  
  return fig
good_isotopes = [index for index in df.index if df.loc[index][0] > 0]

In [25]:
isotopes = widgets.TagsInput(
    placeholder='Type Isotope(s) Here',
    allowed_tags = good_isotopes,
    allow_duplicates = False
)
from IPython.display import display
button = widgets.Button(description="Click Here to Plot")
output = widgets.Output()
def plot_isotopes(b):
    with output:
        plot(isotopes.value).show(renderer="notebook")
button.on_click(plot_isotopes)
isotopes_label = widgets.Label(value="Type Isotope(s) in Box Then Hit Enter")
widgets.VBox([isotopes_label, isotopes, button, output])